In [1]:
import os
import json
import torch
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm
from abc import ABC, abstractmethod
from transformers import AutoProcessor, AutoModel, CLIPModel

import screen_setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ============================================================
#  Base Abstract Class
# ============================================================
class FeatureExtractor(ABC):
    def __init__(self, model_name: str, size: int = 224):
        self.model_name = model_name
        self.model = None
        self.processor = None
        self.size = size
        self.load_model()

    @abstractmethod
    def load_model(self):
        pass

    @abstractmethod
    def extract_features(self, img_np):
        pass

    def process_partial(self, framedata, cross=True):
        """Extract features for a batch of frames (no stacking)."""
        if self.model is not None:
            self.model.eval()
        features = []
        # print(framedata)

        with torch.no_grad():
            for frame in tqdm(framedata, desc=f"Extracting {self.model_name} features (partial)", leave=False):
                img_np = screen_setup.preprocess_image(frame, cross=cross, force_size=self.size)
                # convert from BGR to RGB
                img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

                feat = self.extract_features(img_np)
                features.append(feat)

        return np.stack(features)


# ============================================================
#  Generic Hugging Face Vision Model Extractor
# ============================================================
class HFVisionFeatureExtractor(FeatureExtractor):
    def load_model(self):
        print(f"Loading model: {self.model_name}")
        self.processor = AutoProcessor.from_pretrained(self.model_name)

        self.model = AutoModel.from_pretrained(self.model_name).to(device)

    def extract_features(self, img_np):
        img_pil = Image.fromarray(np.uint8(img_np))
        inputs = self.processor(images=img_pil, return_tensors="pt").to(device)

        if "clip" in self.model_name.lower():
            feats = self.model.get_image_features(pixel_values=inputs["pixel_values"])
        elif "resnet" in self.model_name.lower():
            outputs = self.model(**inputs, output_hidden_states=True)
            features = outputs.hidden_states[-1]
            features = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1))
            feats = features.view(features.size(0), -1)
        else:
            outputs = self.model(**inputs)
            if hasattr(outputs, "image_embeds"):
                feats = outputs.image_embeds
            # Handle pure vision encoders (ViT, DINO, ConvNeXt)
            elif hasattr(outputs, "pooler_output"):
                feats = outputs.pooler_output
            elif hasattr(outputs, "last_hidden_state"):
                feats = outputs.last_hidden_state.mean(dim=1)
            else:
                raise ValueError(f"Unsupported output type for model: {self.model_name}")

            feats = feats / feats.norm(p=2, dim=-1, keepdim=True)
        return feats.cpu().numpy().squeeze()

class FlatImageFeatureExtractor(FeatureExtractor):
    """Feature extractor that resizes the image to (size, size), flattens. Handles color or grayscale."""
    def __init__(self, model_name="raw_image", size=224):
        super().__init__(model_name)
        self.size = size
        self.model = None

    def load_model(self):
        # No model to load
        pass

    def extract_features(self, img_np):
        # img_np: HxWxC numpy array (expects HWC, uint8 or float)
        # Resize to (size, size)
        img_resized = cv2.resize(img_np, (self.size, self.size), interpolation=cv2.INTER_LINEAR)

        if img_resized.ndim == 2:
            # Grayscale image, shape (size, size)
            flat = img_resized.flatten().astype(np.float32)
        elif img_resized.ndim == 3:
            # Color image, shape (size, size, C)
            # Flatten all channels (row-major, then channel per pixel)
            flat = img_resized.flatten().astype(np.float32)
        else:
            raise ValueError(f"Unsupported image shape after resize: {img_resized.shape}")

        return flat


# ============================================================
#  Adversarial Training Resnet50
# ============================================================

import at_resnet50
import dill
import torch

ch = torch
class InputNormalize(ch.nn.Module):
    '''
    A module (custom layer) for normalizing the input to have a fixed 
    mean and standard deviation (user-specified).
    '''
    def __init__(self, new_mean, new_std):
        super(InputNormalize, self).__init__()
        new_std = new_std[..., None, None]
        new_mean = new_mean[..., None, None]

        self.register_buffer("new_mean", new_mean)
        self.register_buffer("new_std", new_std)

    def forward(self, x):
        x = ch.clamp(x, 0, 1)
        x_normalized = (x - self.new_mean)/self.new_std
        return x_normalized
class ATResnet50(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = at_resnet50.resnet50(pretrained=False)
        self.normalizer = InputNormalize(new_mean=torch.tensor([0.485, 0.456, 0.406]), new_std=torch.tensor([0.229, 0.224, 0.225]))

    def forward(self, x, with_latent=False):
        x = self.normalizer(x)
        x, latent = self.model(x, with_latent=with_latent)
        return x, latent


at_resnet_50 = ATResnet50()
checkpoint = torch.load("_dataset_zip/imagenet_l2_3_0.pt", pickle_module=dill, map_location=torch.device('cpu'))
# Makes us able to load models saved with legacy versions
state_dict_path = 'model'
if not ('model' in checkpoint):
    state_dict_path = 'state_dict'
sd = checkpoint[state_dict_path]
sd = {k[len('module.'):]:v for k,v in sd.items() if 'attacker' not in k}
at_resnet_50.load_state_dict(sd)
at_resnet_50 = at_resnet_50.to(device)

class ATResnet50FeatureExtractor(FeatureExtractor):
    def __init__(self, *args, **kwargs):
        if "model_name" not in kwargs:
            kwargs["model_name"] = "at_resnet_50"
        super().__init__(*args, **kwargs)
        # at_resnet_50 is globally available from the top initialization; else, could instantiate here
        self.model = at_resnet_50
        self.model.eval()

    def load_model(self):
        # Model is already loaded in __init__
        pass

    def extract_features(self, img_np):
        # img_np is in RGB, 0-255
        # Convert to torch tensor and normalize to [0, 1]
        img_t = torch.from_numpy(img_np).float() / 255.0
        img_t = img_t.permute(2, 0, 1).unsqueeze(0).to(device)  # (1, 3, H, W)

        # No gradients needed
        with torch.no_grad():
            output, latent = self.model(img_t, with_latent=True)
        # latent is the penultimate features (before fc)
        feats = latent
        return feats.cpu().numpy().squeeze()



/home/zaho/orcd/pool/seeg_rsvp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:

# ============================================================
#  Main Pipeline (chunked feature extraction)
# ============================================================
def process_imagebank(extractorss, imagebank_dirs, imagebank_root="imagebank/", chunk_size=5000):

    for imagebank_dir in imagebank_dirs:
        print(f"\n📁 Processing: {imagebank_dir}")
        # Try framedata_features.json, then framedata.json, then template.json
        for fname in ["framedata_features.json", "framedata.json", "template.json"]:
            path_json = os.path.join(imagebank_root, imagebank_dir, fname)
            if os.path.exists(path_json):
                break
        else:
            raise FileNotFoundError(f"No framedata JSON found in {os.path.join(imagebank_root, imagebank_dir)}")
        with open(path_json, "r") as f:
            framedata = json.load(f)['framedata']

        # Remove all repeats
        unique_paths = set()
        unique_framedata = []
        for frame in framedata:
            if frame['image_path'] not in unique_paths:
                unique_paths.add(frame['image_path'])
                unique_framedata.append(frame)
        framedata = unique_framedata
        # Save the current framedata as framedata_features.json in the appropriate imagebank_dir
        framwdata_features_path = os.path.join(imagebank_root, imagebank_dir, "framedata_features.json")
        with open(framwdata_features_path, "w") as f_out:
            json.dump({"framedata": framedata}, f_out, indent=2)

        for extractor in extractors:
            # sanitize name for filesystem
            safe_name = extractor.model_name.replace("/", "_")
            save_path = os.path.join(imagebank_root, imagebank_dir, f"features_{safe_name}.npy")
            if os.path.exists(save_path):
                print(f"→ Skipping {extractor.model_name} features for {imagebank_dir} because they already exist")
                continue

            # Chunked saving
            print(f"→ Extracting {extractor.model_name} features in chunks of {chunk_size} ...")
            n_images = len(framedata)
            part_paths = []
            for part_idx, start_idx in enumerate(range(0, n_images, chunk_size)):
                partpath = os.path.join(imagebank_root, imagebank_dir, f"features_{safe_name}_part{part_idx+1}.npy")
                if os.path.exists(partpath):
                    print(f"→ Skipping {extractor.model_name} features for {imagebank_dir} because they already exist")
                    continue
                end_idx = min(start_idx + chunk_size, n_images)
                feats = extractor.process_partial(framedata[start_idx:end_idx])
                np.save(partpath, feats)
                part_paths.append(partpath)
                print(f"    Saved chunk {part_idx+1} [{start_idx}:{end_idx}] to {partpath}")
                
            # Combine
            print(f"→ Combining {len(part_paths)} parts into one final npy...")
            all_parts = [np.load(pp) for pp in part_paths]
            full_feats = np.concatenate(all_parts)
            np.save(save_path, full_feats)
            print(f"✅ Saved combined: {save_path}")

            # Clean up part files
            for pp in part_paths:
                os.remove(pp)

imagebank_root="imagebank/"
imagebank_dirs = [
    d for d in os.listdir(imagebank_root)
    if os.path.isdir(os.path.join(imagebank_root, d))
]
# imagebank_dirs = ["trial_s1_n480_x4_on100-100_off125-175"]


model_list = [
    "openai/clip-vit-base-patch32",
    "facebook/dino-vits16",
    "google/vit-base-patch16-224",
    "microsoft/resnet-50",
]
extractors = [HFVisionFeatureExtractor(name) for name in model_list]
# extractors.append(FlatImageFeatureExtractor())
extractors += [ATResnet50FeatureExtractor()]

process_imagebank(extractors, imagebank_dirs)


Loading model: openai/clip-vit-base-patch32


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading model: facebook/dino-vits16


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits16 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model: google/vit-base-patch16-224


...

    Saved chunk 1 [0:960] to imagebank/trial_s3_n960_x2_on100-100_off125-175/features_google_vit-base-patch16-224_part1.npy
→ Combining 1 parts into one final npy...
✅ Saved combined: imagebank/trial_s3_n960_x2_on100-100_off125-175/features_google_vit-base-patch16-224.npy
→ Extracting microsoft/resnet-50 features in chunks of 5000 ...


    Saved chunk 1 [0:960] to imagebank/trial_s3_n960_x2_on100-100_off125-175/features_microsoft_resnet-50_part1.npy
→ Combining 1 parts into one final npy...
✅ Saved combined: imagebank/trial_s3_n960_x2_on100-100_off125-175/features_microsoft_resnet-50.npy
→ Extracting at_resnet_50 features in chunks of 5000 ...


    Saved chunk 1 [0:960] to imagebank/trial_s3_n960_x2_on100-100_off125-175/features_at_resnet_50_part1.npy
→ Combining 1 parts into one final npy...
✅ Saved combined: imagebank/trial_s3_n960_x2_on100-100_off125-175/features_at_resnet_50.npy

📁 Processing: ILSVRC2012_img_val
→ Extracting openai/clip-vit-base-patch32 features in chunks of 5000 ...


    Saved chunk 1 [0:5000] to imagebank/ILSVRC2012_img_val/features_openai_clip-vit-base-patch32_part1.npy


Extracting openai/clip-vit-base-patch32 features (partial):   9%|▉         | 466/5000 [01:00<13:08,  5.75it/s]